### Below is a concise yet friendly explanation of the attention mechanism for assignment introduction:

#### Attention Mechanism (Adapted from “Attention Is All You Need”)

The attention mechanism, introduced in Attention Is All You Need (Vaswani et al., 2017), processes inputs represented as vectors (each row is a token embedding of dimension $𝐷$). We compute three sets of vectors: queries (Q), keys (K), and values (V). The core steps are:

1. **Linear Transformations:**  

Let $X$ be the input matrix, where each of the rows corresponds to a token in the input sequence, and each row is a $d$-dimensional embedding vector.

To compute attention, we first project $X$ into three different representations using learned weight matrices:

Each input vector is transformed into $Q$, $K$, and $V$ using learnable weights.

$$
\begin{aligned}
Q_i &= X W_i^Q, \\
K_i &= X W_i^K, \\
V_i &= X W_i^V.
\end{aligned}
$$

Each head \(i\) has its own learnable parameters $W_i^Q$, $W_i^K$, and $W_i^V$, which transform the input into queries, keys, and values, respectively.


2. **Scaled Dot-Product Attention:**  

\begin{equation}
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^\top}{\sqrt{d_k}}\right)V.
\end{equation}

Here, $QK^T$ produces a matrix of scores that measures how relevant each “query” position is to every “key” position. 

The softmax function converts these scores into attention weights (non-negative values that sum to 1 across each row).

These weights are then used to combine the values 𝑉 to produce the final output.




3. **Multi-Head Attention:**  

$$
\text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1, \dots, \text{head}_h)
$$

$$
\text{where } \text{head}_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V).
$$


Multiple attention heads allow the model to attend to different aspects of the input simultaneously. Their outputs are concatenated and linearly transformed to produce the final result.



*Note:* In this assignment, you are only required to experiment with the provided $Q$, $K$, and $V$ matrices to perform the matrix multiplication

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Data size
N = 5
D = 6

X = [[ 0.7, -0.8, -1.2,  -1.,  -0., -0.3],
     [ 2.7,  0.1,  1.6,  1.8,  1.5,  0.3],
     [ 0.1,  2.6, -0.1, -1.3, -0.5, -0.7],
     [ 1.1,  1.5,   1., -0.5,  0.4,  0.4],
     [-0.7, -0.7,  0.7, -1.5, -0.8,  1. ]]

Wq = [[-1.7,  1.6,  0.9, -0.5,  0.4,  -1.],
      [-0.4,  1. , -0.3,  1. ,  0.5,  1.1],
      [ 0.4, -0.9,  -1.,  0.5, -1.4,  0. ],
      [ 0.3,  1.4, -1.2,  0.2,  0.1,  1.6],
      [-0.8,  0.8, -0.7, -1.3,  0.3,  0.8],
      [ 1.1,  0.3, -1.5, -2.3,  2.2, -0.7]]

Wk = [[ 0.3, -0.4, -1.3,  0.3, -1.7,  1.1],
      [-2.3, -1.1,  0.6, -1.2,  2.2,  0.3],
      [ 1.1, -0.4, -0.5,  1.9, -1.1, -1.2],
      [-0.4,  1. , -1.7,  0. , -3.3, -1.4],
      [-0.9, -1.1, -1. ,  1.4,  1.3,  1.2],
      [-0.7,  0.4,  0.4, -1.4, -0.2, -0.5]]

Wv = [[-0.1,  0.7,  1. , -0.1,  1.6,  0.9],
      [ 0.4, -1. , -0.7, -0.6, -0.9, -0.1],
      [-0.4,  0.5, -1.4,  0.1,  0.6,  0.4],
      [ 1.4, -1.3, -1.3, -0.6,  1.6, -0.2],
      [-0.4, -0.6, -1.4, -1. ,  0.4, -0.8],
      [ 0.2,  0.5,  0.4, -0.5,  1.4,  2.3]]
X = np.array(X)
Wq = np.array(Wq)
Wk = np.array(Wk)
Wv = np.array(Wv)

### (a) Implement the self-attention operation

In [3]:
def softmax(x, axis=-1):
    """
    Stable softmax function with numerical stability optimization
    Args:
        x: Input array (numpy ndarray)
        axis: Axis along which to compute softmax (default: -1)
    Returns:
        Probability distribution array along specified axis
    """
    max_scores = np.max(x, axis=-1, keepdims=True)
    exp_scores = np.exp(x - max_scores)
    sum_exp = np.sum(exp_scores, axis=-1, keepdims=True)
    
    return exp_scores / sum_exp

def self_attention(X, Wq, Wk, Wv):
    """
    Transformer self-attention mechanism implementation
    Args:
        X: Input matrix [num_samples, dim_input]
        Wq: Query weight matrix [dim_input, d_k]
        Wk: Key weight matrix [dim_input, d_k]
        Wv: Value weight matrix [dim_input, d_k]
    Returns:
        output: Context vectors [num_samples, d_k]
        attn_weights: Attention probabilities [num_samples, num_samples]
    """
    
    num_samples, dim_input = X.shape
    d_k = Wq.shape[-1]
    
    # Phase 1: Linear Transformations
    Q = X @ Wq # [num_samples, d_k]
    K = X @ Wk # [num_samples, d_k]
    V = X @ Wv # [num_samples, d_k]
    
    # Phase 2: Scaled Dot-Product Attention
    # Compute attention scores and weights
    d_k = Q.shape[-1]
    scores = (Q @ K.T) / np.sqrt(d_k) # [num_samples, num_samples]
    attn_weights = softmax(scores, axis=-1)
    
    # Phase 3: Output calculation 
    # Weighted sum of value vectors
    attn_weights = softmax(scores, axis=-1)
    output = attn_weights @ V # [num_samples, d_k]
    return output, attn_weights

In [4]:
# Compute the output
output, attention_weights = self_attention(X, Wq, Wk, Wv)

# Print in a nice format
np.set_printoptions(precision=1)
print("Self-Attention Output:\n", output)
print("Self-Attention Weights:\n", attention_weights)

Self-Attention Output:
 [[-0.7 -0.9  0.5  0.1 -5.5 -1.1]
 [-0.4  0.6  0.6 -0.6  2.   0.5]
 [ 0.3 -0.1 -2.4 -1.9  4.9  1.8]
 [-0.7 -0.7  0.4 -0.1 -4.5 -0.7]
 [-2.   3.3  2.1  1.6 -1.3  2.8]]
Self-Attention Weights:
 [[4.9e-07 4.0e-14 9.9e-01 1.7e-05 6.1e-03]
 [4.8e-01 3.6e-01 1.5e-01 2.1e-02 4.0e-04]
 [1.9e-02 5.8e-01 9.4e-02 2.8e-01 3.2e-02]
 [3.0e-02 1.2e-02 8.5e-01 9.9e-02 1.1e-02]
 [4.7e-04 7.3e-03 1.6e-02 4.0e-02 9.4e-01]]


### (b) Implement multi-head attention, using the previously implemented function

In [5]:
def multi_head_attention(X, Wq, Wk, Wv, H):
    """
    Implements multi-head attention mechanism in transformer
    Args:
        X: Input array [num_samples, dim_input]
        Wq: Query weight array [dim_input, dim_input] (concatenated heads)
        Wk: Key weight array [dim_input, dim_input]
        Wv: Value weight array [dim_input, dim_input]
        H: Number of attention heads
    Returns:
        concatenated: Combined output array [num_samples, dim_input]
        all_attn_weights: List of attention matrices per head
    """
    
    # Phase 1: Initialization
    num_samples, dim_input = X.shape
    dim_per_head = dim_input // H # Features per attention head
    outputs = []
    all_attn_weights = []
    
    # Phase 2: Head Processing Loop
    for h in range(H):
        # Calculate slice indices for current head
        start = h * dim_per_head
        end = start + dim_per_head
        
        # Slice columns for each head's weights
        Wq_h = Wq[:, start:end]  # [dim_input, dim_per_head]
        Wk_h = Wk[:, start:end]
        Wv_h = Wv[:, start:end]
        
        # Process single attention head
        # Pass weights without transposing (handled in self_attention)
        head_output, attn_weights = self_attention(X, Wq_h, Wk_h, Wv_h) 

        outputs.append(head_output) # [num_samples, dim_per_head]
        all_attn_weights.append(attn_weights) # [num_samples, num_samples]
        
    # Phase 3: Output Concatenation    
    concatenated = np.concatenate(outputs, axis=-1) # [num_samples, dim_input]
    
    return concatenated, all_attn_weights
    

In [6]:
# Compute multi-head attention
H = 1
attention_output,_ = multi_head_attention(X, Wq, Wk, Wv, H)
print("Multi-Head Attention Output, H=1:\n", attention_output)

Multi-Head Attention Output, H=1:
 [[-0.7 -0.9  0.5  0.1 -5.5 -1.1]
 [-0.4  0.6  0.6 -0.6  2.   0.5]
 [ 0.3 -0.1 -2.4 -1.9  4.9  1.8]
 [-0.7 -0.7  0.4 -0.1 -4.5 -0.7]
 [-2.   3.3  2.1  1.6 -1.3  2.8]]


In [7]:
# Compute multi-head attention
H = 3
attention_output,_ = multi_head_attention(X, Wq, Wk, Wv, H)
print("Multi-Head Attention Output, H=3:\n", attention_output)

Multi-Head Attention Output, H=3:
 [[-0.7 -0.9  0.7  0.2 -1.5  2.9]
 [-1.1  0.9 -3.9 -2.9 -3.  -0.5]
 [-0.7 -0.9  1.6  1.2  5.8  1.5]
 [-0.7 -0.7 -3.8 -2.8 -5.1 -0.9]
 [-0.2  0.3  1.   0.2 -1.3  3. ]]


#### We can confirm attention score correctness through PyTorch's built-in MultiheadAttention module by aligning implementations.
Note: This verification section isn't part of formal grading

In [8]:
# PyTorch Implementation Comparison
# To validate custom multi-head attention implementation 

import torch
import torch.nn as nn

X_torch = torch.tensor(X, dtype=torch.float32).unsqueeze(1)
Wq_torch = torch.tensor(Wq, dtype=torch.float32)
Wk_torch = torch.tensor(Wk, dtype=torch.float32)
Wv_torch = torch.tensor(Wv, dtype=torch.float32)


# PyTorch comparison for H=1
mha_H1 = nn.MultiheadAttention(6, 1, batch_first=False)
with torch.no_grad():
    mha_H1.in_proj_weight.data = torch.cat([Wq_torch.T, Wk_torch.T, Wv_torch.T], dim=0)
    mha_H1.out_proj.weight.data = torch.eye(6)
    mha_H1.out_proj.bias.data.zero_()
output_torch_H1, _ = mha_H1(X_torch, X_torch, X_torch)
output_torch_H1 = output_torch_H1.detach().numpy()

print("H=1 Attention Weights (Head 1):\n", output_torch_H1)


# PyTorch comparison for H=3
mha_H3 = nn.MultiheadAttention(6, 3, batch_first=False)
with torch.no_grad():
    mha_H3.in_proj_weight.data = torch.cat([Wq_torch.T, Wk_torch.T, Wv_torch.T], dim=0)
    mha_H3.out_proj.weight.data = torch.eye(6)
    mha_H3.out_proj.bias.data.zero_()
output_torch_H3, _ = mha_H3(X_torch, X_torch, X_torch)
output_torch_H3 = output_torch_H3.detach().numpy()

print("H=3 Attention Weights (Head 3):\n", output_torch_H3)


H=1 Attention Weights (Head 1):
 [[[-0.7 -0.9  0.5  0.1 -5.5 -1.1]]

 [[-0.4  0.6  0.6 -0.6  2.   0.5]]

 [[ 0.3 -0.1 -2.4 -1.9  4.9  1.8]]

 [[-0.7 -0.7  0.4 -0.1 -4.5 -0.7]]

 [[-2.   3.3  2.1  1.6 -1.3  2.8]]]
H=3 Attention Weights (Head 3):
 [[[-0.7 -0.9  0.7  0.2 -1.5  2.9]]

 [[-1.1  0.9 -3.9 -2.9 -3.  -0.5]]

 [[-0.7 -0.9  1.6  1.2  5.8  1.5]]

 [[-0.7 -0.7 -3.8 -2.8 -5.1 -0.9]]

 [[-0.2  0.3  1.   0.2 -1.3  3. ]]]


### (c+d) Provide the answers/explanations requested in the problem sheet:
1. Why the results are different?
2. What happens if you change the order of two inputs=

In [9]:
# Swap first two input sequences (rows 0 and 1)
X_swapped = X.copy() 
X_swapped[[0,1],:] = X_swapped[[1,0],:]

output_original_H1, _ = multi_head_attention(X, Wq, Wk, Wv, H=1)

# Swapped order output
output_swapped_H1, _ = multi_head_attention(X_swapped, Wq, Wk, Wv, H=1)

print("H=1 Comparison:")
print("Original Order Output :\n", output_original_H1)
print("Swapped Order Output :\n", output_swapped_H1)


H=1 Comparison:
Original Order Output :
 [[-0.7 -0.9  0.5  0.1 -5.5 -1.1]
 [-0.4  0.6  0.6 -0.6  2.   0.5]
 [ 0.3 -0.1 -2.4 -1.9  4.9  1.8]
 [-0.7 -0.7  0.4 -0.1 -4.5 -0.7]
 [-2.   3.3  2.1  1.6 -1.3  2.8]]
Swapped Order Output :
 [[-0.4  0.6  0.6 -0.6  2.   0.5]
 [-0.7 -0.9  0.5  0.1 -5.5 -1.1]
 [ 0.3 -0.1 -2.4 -1.9  4.9  1.8]
 [-0.7 -0.7  0.4 -0.1 -4.5 -0.7]
 [-2.   3.3  2.1  1.6 -1.3  2.8]]


In [10]:
output_original_H3, _ = multi_head_attention(X, Wq, Wk, Wv, H=3)

# Swapped order output
output_swapped_H3, _ = multi_head_attention(X_swapped, Wq, Wk, Wv, H=3)

print("H=3 Comparison:")
print("Original Order Output :\n", output_original_H3)
print("Swapped Order Output :\n", output_swapped_H3)

H=3 Comparison:
Original Order Output :
 [[-0.7 -0.9  0.7  0.2 -1.5  2.9]
 [-1.1  0.9 -3.9 -2.9 -3.  -0.5]
 [-0.7 -0.9  1.6  1.2  5.8  1.5]
 [-0.7 -0.7 -3.8 -2.8 -5.1 -0.9]
 [-0.2  0.3  1.   0.2 -1.3  3. ]]
Swapped Order Output :
 [[-1.1  0.9 -3.9 -2.9 -3.  -0.5]
 [-0.7 -0.9  0.7  0.2 -1.5  2.9]
 [-0.7 -0.9  1.6  1.2  5.8  1.5]
 [-0.7 -0.7 -3.8 -2.8 -5.1 -0.9]
 [-0.2  0.3  1.   0.2 -1.3  3. ]]


1. Each head learns to represent different relationships in the data: the attention weights are computed separately and the outputs are concatenated then pulled back.

2. Swapping the first two inputs only swaps their positions in the output: for attention output matrices, rows 1-2 swapped compared to original The attention weights adjust to the new input order, reflecting permutation equivariance. The output differences will be non-zero but structured.